In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.layers import LSTM
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Dense, Flatten, Input, LSTM, Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

# CSV 파일 로드
df = pd.read_csv('ae_data.csv')

# 데이터 정보 확인
df.info()

# 기본 통계량 확인
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18398 entries, 0 to 18397
Data columns (total 63 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    18398 non-null  object 
 1   y       18398 non-null  int64  
 2   x1      18398 non-null  float64
 3   x2      18398 non-null  float64
 4   x3      18398 non-null  float64
 5   x4      18398 non-null  float64
 6   x5      18398 non-null  float64
 7   x6      18398 non-null  float64
 8   x7      18398 non-null  float64
 9   x8      18398 non-null  float64
 10  x9      18398 non-null  float64
 11  x10     18398 non-null  float64
 12  x11     18398 non-null  float64
 13  x12     18398 non-null  float64
 14  x13     18398 non-null  float64
 15  x14     18398 non-null  float64
 16  x15     18398 non-null  float64
 17  x16     18398 non-null  float64
 18  x17     18398 non-null  float64
 19  x18     18398 non-null  float64
 20  x19     18398 non-null  float64
 21  x20     18398 non-null  float64
 22

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
count,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,...,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000
mean,0.006740,0.011824,0.157986,0.569300,-9.958345,0.006518,2.387533,0.001647,-0.004125,-0.003056,...,0.380519,0.360246,0.173708,2.379154,9.234953,0.233493,-0.001861,-0.061522,0.001258,0.001033
std,0.081822,0.742875,4.939762,5.937178,131.033712,0.634054,37.104012,0.108870,0.075460,0.156047,...,6.211598,14.174273,3.029516,67.940694,81.274103,2.326838,0.048732,10.394085,0.004721,0.032120
min,0.000000,-3.787279,-17.316550,-18.198509,-322.781610,-1.623988,-279.408440,-0.429273,-0.451141,-0.120087,...,-187.943440,-1817.595500,-8.210370,-230.574030,-269.039500,-12.640370,-0.149790,-100.810500,-0.012229,0.000000
25%,0.000000,-0.405681,-2.158235,-3.537054,-111.378372,-0.446787,-24.345268,-0.058520,-0.051043,-0.059966,...,-3.672684,-1.928166,0.487780,-40.050046,-45.519149,-1.598804,0.000470,0.295023,-0.001805,0.000000
50%,0.000000,0.128245,-0.075505,-0.190683,-14.881585,-0.120745,10.528435,-0.009338,-0.000993,-0.030057,...,0.294846,0.143612,0.702299,17.471317,1.438806,0.085826,0.012888,0.734591,0.000710,0.000000
75%,0.000000,0.421222,2.319297,3.421223,92.199134,0.325152,32.172974,0.060515,0.038986,0.001990,...,5.109543,3.230770,2.675751,44.093387,63.209681,2.222118,0.020991,1.266506,0.004087,0.000000
max,1.000000,3.054156,16.742105,15.900116,334.694098,4.239385,96.060768,1.705590,0.788826,4.060033,...,14.180588,11.148006,6.637265,287.252017,252.147455,6.922008,0.067249,6.985460,0.020510,1.000000


In [4]:
# 결측치 확인
df.isnull().sum()

time    0
y       0
x1      0
x2      0
x3      0
       ..
x57     0
x58     0
x59     0
x60     0
x61     0
Length: 63, dtype: int64

In [5]:
# 결측치 제거, 확인
df = df.dropna()
df.isnull().sum()

time    0
y       0
x1      0
x2      0
x3      0
       ..
x57     0
x58     0
x59     0
x60     0
x61     0
Length: 63, dtype: int64

In [6]:
# 컬렴명 확인
df.columns

Index(['time', 'y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9',
       'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19',
       'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29',
       'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39',
       'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49',
       'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59',
       'x60', 'x61'],
      dtype='object')

In [7]:
# 레이블 갯수 확인 (숫자)
df['time'].value_counts()

time
5/1/99 0:00     1
5/20/99 1:14    1
5/20/99 1:26    1
5/20/99 1:24    1
5/20/99 1:22    1
               ..
5/10/99 7:44    1
5/10/99 7:46    1
5/10/99 7:48    1
5/10/99 7:50    1
5/29/99 0:06    1
Name: count, Length: 18398, dtype: int64

In [11]:
# train/test 분할

from sklearn.model_selection import train_test_split

X = df.drop('time', axis=1).values # pandas->numpy하며 time 레이블을 제거
y = df['time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



In [12]:
# split_sequence 정의

def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        # 시퀀스 끝을 넘어가면 중단
        if end_ix > len(sequence) - 1:
            break
        # 입력 시퀀스와 출력 시퀀스 분리
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)



In [13]:
# n_steps 설정
n_steps = 62  # 모델 입력으로 사용할 시퀀스 길이

# 데이터 분할
X_train, y_train = split_sequence(X_train, n_steps)
X_test, y_test = split_sequence(X_test, n_steps)

In [15]:
# 모델

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM, Dense, Flatten

def create_cnn_lstm_model(input_shape, output_shape):
    model = Sequential([
        # 1D Convolutional Layer
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        
        # 추가 Convolutional Layer
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        
        # Flatten layer to reshape data for LSTM input
        Flatten(),
        
        # LSTM Layer
        LSTM(50, activation='relu'),
        
        # Fully connected layers
        Dense(64, activation='relu'),
        Dense(output_shape, activation='sigmoid')  # 이진 분류를 위한 sigmoid
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [16]:
# X_train과 X_test의 차원 조정 (features=1 추가)
X_train_seq = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # (samples, time_steps, features)
X_test_seq = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))      # (samples, time_steps, features)

print("X_train_seq shape:", X_train_seq.shape)
print("y_train shape:", y_train.shape)
print("X_test_seq shape:", X_test_seq.shape)
print("y_test shape:", y_test.shape)

# 모델 학습
model.fit(X_train_seq, y_train, epochs=100, batch_size=32, validation_data=(X_test_seq, y_test))

ValueError: cannot reshape array of size 56337664 into shape (14656,62,1)

In [14]:
# 데이터 준비 후 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100


ValueError: in user code:

    File "c:\Users\calix\anaconda3\envs\DLvirtual\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\calix\anaconda3\envs\DLvirtual\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\calix\anaconda3\envs\DLvirtual\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\calix\anaconda3\envs\DLvirtual\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\calix\anaconda3\envs\DLvirtual\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\calix\anaconda3\envs\DLvirtual\lib\site-packages\keras\engine\input_spec.py", li

In [10]:
# 모델 성능 평가

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

NameError: name 'model' is not defined